# Chapter 8: Optimization & Gradient Descent

**Jonny Yu, March 2024**

## Lecture Learning Objectives
<hr>

- Explain the difference between a model, loss function, and optimization algorithm in the context of machine learning
- Explain how the gradient descent algorithm works
- Apply gradient descent to linear and logistic regression
- Use `scipy.optimize.minimize()` to minimize a function

## Imports
<hr>

In [4]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.metrics import mean_squared_error, mean_absolute_error
from utils.plotting import *

## 1. Optimization and Machine Learning
<hr>

- In data science, we optimize a lot of stuff, e.g., in linear regression we optimize for the intercept and coefficients, in clustering we optimize clusters, in neural networks we optimize the weights in our network (more on that in a later lecture), etc.
- In one sentence: optimization simply refers to minimizing/maximizing a function, e.g.:
    - What value of $x$ minimizes the function $f(x) = (x-2)^2 + 5$? What is the minimum value?
    - *Answers*: $x=2$, and $f(x)=5$.

- You can start to think of ML as a 3-step process:
    1. **Choose your model**: controls the space of possible functions that map $X$ to $y$ (e.g., a linear model can only learn linear functions)
    2. **Choose your loss function**: tells us how to compare these various functions (e.g., is $y=5 + 2x_1+3x_2$ a better model than $y=1 + 10x_1-x_2$?)
    3. **Choose your optimization algorithm**: finds the minimum of the loss function (e.g., what is the optimum value of $w_0$ and $w_1$ in $y=w_0 + w_1x$?)
- In this lecture we'll be taking a visual approach to understanding optimization and gradient descent (an optimization algorithm)

## 2. Loss Functions
<hr>

- A loss function is what we use to map the performance of our model to a real number and it's the thing we want to optimize!
- For example, here's the mean squared error (a common loss function):

$$f(y,\hat{y})=\frac{1}{n}\sum^{n}_{i=1}(\hat{y_i}-y_i)^2$$

- Where $y_i$ is the actual response and $\hat{y_i}$ is the predicted response
- Consider a simple linear regression model $\hat{y_i} = w_0 + w_1x_i$, then our loss function is:

$$f(y,x,w)=\frac{1}{n}\sum^{n}_{i=1}((w_0 + w_1x_i)-y_i))^2$$

- The optimization problem here is to find the values of $w_0$ and $w_1$ that minimizes the MSE

## 3. Optimizing Linear Regression
<hr>

- I'm going to build up the intuition for optimization in a practical and visual way with the help of our old friend linear regression
- If you'd prefer a more mathematical approach I encourage you to check out Mike's lecture on Youtube [Thinking about Optimization](https://www.youtube.com/watch?v=bzj1L997uT8&feature=youtu.be&t=67) or [Chapter 7 of Mathematics for Machine Learning, by Deisenroth et al.](https://mml-book.github.io/book/mml-book.pdf)
- We'll use a dataset of [Pokemon](https://www.pokemon.com/us/) "attack" and "defense" stats to do this, I'm going to start with just 10 observations:

> Throughout this lecture, I'm leveraging plotting scripts I imported from `utils.plotting`. I abstracted the code out of the notebook to avoid cluttering the lecture notes and how I made these plots is not important, but feel free to check out the source code if you wish!

>ps. the most powerful pokemon of them all, **Poketom**, is unfortunately not in this dataset:


- Recall simple linear regression: $\hat{y_i} = w_0 + w_1x_i$ (where  $w_0$ is the intercept and $w_1$ is the slope coefficient)
- If we assume ($w_0$, $w_1$) = (10, 0.5) we would have:

- Let's plot that result:

- The fit is not very good... We need to optimize it!
- A **loss function** quantifies the fit of our model and we want to find the parameters of our model that minimize the loss function
- We'll use mean-squared-error (MSE) as our loss function:

$$f(w)=\frac{1}{n}\sum^{n}_{i=1}((w_0 + w_1x_i)-y_i))^2$$

- Where $n$ is the number of data points we have (10 in our case).
- We'll use the sklearn function `mean_squared_error()` which I imported at the top of the notebook to calculate MSE for us:

- So this is the mean error of every single training example
- For now, let's assume the intercept is 0 ($w_0 = 0$) and **just focus on optimizing the slope** ($w_1$)
- One thing we could do is try many different values for the slope and find the one that minimizes the MSE:

- It looks like a slope of 0.9 gives us the lowest MSE (~184.4)
- But you can imagine that this "grid search" approach quickly becomes computationally intractable as the size of our data set and number of model parameters increases!
- So we need a better way to optimize our parameters... we need an optimization algorithm!
- The one we'll focus on today is **Gradient Descent**

## 4. Gradient Descent With One Parameter
<hr>

- Gradient descent is an optimization algorithm that can help us optimize our loss function
- As the name suggest, we are going to leverage the **gradient** of our loss function to help us optimize our model parameters
- The gradient is just a vector of (partial) derivatives of the loss function w.r.t the model parameters
- Sounds complicated but it's not at all (as I'll hopefully show you)
- In plain English, the gradient will tell us two things:
    1. Which direction to move our parameter in to decrease loss (i.e., should we increase or decrease its value?)
    2. How far to move it (i.e., should we adjust it by 0.1 or 2 or 50 etc.?)

- Let's forget about the intercept now and just work with this simple linear model: $\hat{y_i}=wx_i$
- For our model the loss function has the form:

$$f(w)=\frac{1}{n}\sum^{n}_{i=1}((wx_i)-y_i))^2$$

- The gradient of this function with respect to the parameter $w$ is:

$$\frac{d}{dw}f(w)=\frac{1}{n}\sum^{n}_{i=1}2x_i(wx_i - y_i)$$

- Let's code that up:
- Let's calculate the gradient of our loss function at a slope of $w = 0.5$:

- So this is the average gradient across all training examples and tells us how to adjust $w$ to reduce the MSE loss over all training examples!
- Recall from calculus that the **gradient actually points in the direction of steepest ascent** (read more in [Appendix A](appendixA_gradients.ipynb))
- We want to move in the direction of steepest descent (the negative of the gradient) to reduce our loss
- For example, the above gradient is negative, but we obviously need to **increase** the value of our slope ($w$) to reduce our loss as you can see here:

- The amount we adjust our slope each iteration is controlled by a "learning rate", denoted $\alpha$ (note the minus in the equation below which accounts for the "descent" as discussed above):

$$w_{new} = w - \alpha \times gradient$$

$$w_{new} = w - \alpha \frac{d}{dw}f(w)$$

- $\alpha$ is a hyperparameter that can be optimized, typical values range from 0.001 to 0.9
- With the math out of the way, we're now ready to use gradient descent to optimize our slope
- Typically we stop gradient descent when:
    1. the step size is smaller than some threshold; or,
    2. a certain number of steps is completed
- So the pseudo code for gradient descent boils down to this:

```sh
1. begin with with some arbitrary w
while stopping criteria not met:
    2. calculate mean gradient across all examples
    3. update w based on gradient and learning rate
    4. repeat
```

- Let's go ahead and implement that now:

In [ ]:
def gradient_descent(x, y, w, alpha, ϵ=2e-4, max_iterations=5000, print_progress=10):


gradient_descent(x, y, w=0.5, alpha=0.00001)

- Let's take a look at the journey our slope parameter went on:

In [ ]:
plot_gradient_descent(x, y, w=0.5, alpha=0.00001)

- Let's see what happens if we increase the learning rate:

In [ ]:
gradient_descent(x, y, w=0.5, alpha=0.00005, print_progress=2)

In [ ]:
plot_gradient_descent(x, y, w=0.5, alpha=0.00005)

- Let's increase a little more:

In [ ]:
gradient_descent(x, y, w=0.5, alpha=0.00015)

In [ ]:
plot_gradient_descent(x, y, w=0.5, alpha=0.00015)

- If our learning rate is too high, we'll overshoot and never converge!

In [ ]:
gradient_descent(x, y, w=0.5, alpha=0.00018, max_iterations=4, print_progress=1)

In [ ]:
plot_gradient_descent(x, y, w=0.5, alpha=0.00018, max_iterations=4)

- Cool, we just implemented gradient descent from scratch!
- Now let's try optimizing for two parameters, intercept and slope, simultaneously...

## 5. Gradient Descent With Two Parameters
<hr>

- And then there were two...


- Most of the models you'll be working with will have more than just one parameter to update - neural networks typically have hundreds, thousands, and even millions of parameters!
- Let's extend the above workflow to two parameters, the intercept ($w_0$) and the slope ($w_1$)
- Just to help you get a visual of what's going on, let's take our "grid search approach" and make a plot of it but this time with two parameters:

- Above is the surface of MSE for different values of `intercept` ($w_0$) and `slope` ($w_1$)
- The approach is exactly as before, but we're operating on two parameters now and the gradient of the intercept is a little different to the slope:

$$f(w)=\frac{1}{n}\sum^{n}_{i=1}((w_0 + w_1x)-y_i))^2$$

$$\frac{\partial{}}{\partial{}w_0}f(w)=\frac{1}{n}\sum^{n}_{i=1}2((w_0 + w_1x) - y_i)$$

$$\frac{\partial{}}{\partial{}w_1}f(w)=\frac{1}{n}\sum^{n}_{i=1}2x_i((w_0 + w_1x) - y_i)$$

- Let's define a function that returns the gradient (partial derivatives) of the slope and intercept:

In [ ]:
def gradient(x, y, w):


gradient(x, y, w=[10, 0.5])

- You can already see that our the gradient of our slope is orders of magnitude larger than our intercept... we'll look more at this issue shortly
- For now let's re-write our gradient descent function
- It's almost exactly the same as before, but now we are updating the slope **and** the intercept each iteration:

In [ ]:
def gradient_descent(x, y, w, alpha, ϵ=2e-4, max_iterations=5000, print_progress=10):
    """Gradient descent for optimizing simple linear regression."""


gradient_descent(x, y, w=[10, 0.5], alpha=0.00001)

- Hmm... our algorithm worked but our intercept never changed
- Let's take a look at what happened:

In [ ]:
plot_gradient_descent_2d(x, y, w=[10, 0.5], m_range=np.arange(0, 2.05, 0.05), b_range=np.arange(-30, 31, 2), alpha=0.00001)

- That's because the slope gradient is **wayyyyy** bigger than the intercept gradient
- Let's see what the real value of these params are using sklearn's implementation:

- So we're a bit off here
- We need to get the slope and intercept on the same scale...

### 5.1. Scaling

- There are a few ways we can solve this problem, but the most common way is to simply scale your features before gradient descent

- Now let's check a random gradient:

- Great, they are on a similar scale now, let's retry gradient descent with our scaled data:

- Matches perfectly!

- Once again, changing the learning rate will affect our descent (I added some markers on this plot to show you that we're bouncing back-and-forth):

## 6. Other Optimization Algorithms
<hr>

- When you saw us using gradients earlier on you might have thought, why not just set the derivative to 0 and solve?
- You sure could do this! And in general, if a closed form solution exists for your problem, you should typically use it
- However:
    - Most problems in ML do not have a closed-form solution
    - Even if a closed form solution exists (e.g., linear regression), it can be extremely computationally expensive to compute if your dataset is large (many observations and/or many features)
- In these cases, optimization algorithms like GD are appropriate choices
- In actuality you will almost never use vanilla GD in practice because it's actually very slow (but the intuition behind it forms the basis of tons of optimization algorithms so it's a great place to start learning)
- We'll look at a computationally lighter version of GD next lecture (stochastic gradient descent) and there are also many other algorithms available!
- You can explore the `scipy` function `minimize` to play around with some of these algorithms:

In [ ]:
from scipy.optimize import minimize

- Here was our gradient descent implementation:

- `minimize` takes as argument the function to minimize, the function's gradient and the starting parameter values
- For a linear regression model, the MSE loss and gradient take the general form:

$$f(w)=\frac{1}{n}\sum^{n}_{i=1}(w^Tx_i-y_i)^2$$

$$\frac{\partial{}}{\partial{}f(w)}=\frac{2}{n}\sum^{n}_{i=1}(w^Tx_i-y_i)x_i$$

- I've coded up that up using matrix multiplication (more on that a bit later):

In [ ]:
def mse(w, X, y):



def mse_grad(w, X, y):
    """Gradient of mean squared error."""


- Now we can call `minimize`:

In [ ]:
minimize(mse, w, jac=mse_grad, args=(x, y)).x  # these are the weights, it's a bit confusing because they are called "x"

- There are plenty of methods you can look into (most give the same answers):

In [ ]:
for method in ["CG", "L-BFGS-B", "SLSQP", "TNC"]:
    print(f"Method: {method}, Weights: {minimize(mse, w, jac=mse_grad, args=(x, y), method=method).x}")

## 7. Lecture Exercise: Optimizing Logistic Regression
<hr>

- In this section I'm going to demo optimizing a Logistic Regression problem to drive home some of the points we learned in this lecture
- I'm going to sample 70 "legendary" (which are typically super-powered) and "non-legendary" pokemon from our dataset

In [ ]:
df.head(10)

- We'll be using the "trick of ones" to help us implement these computations efficiently
- For example, if we have a simple linear regression model with an intercept and a slope:

$$\hat{y} = \boldsymbol{w^T}\boldsymbol{x} = w_0\times{}1 + w_1\times{}x$$

- Let's represent that in matrix form:

$$\begin{bmatrix} y_1 \\ y_2 \\ \vdots \\ y_n \end{bmatrix}=\begin{bmatrix} 1 & x_1 \\ 1 & x_2 \\ \vdots & \vdots \\ 1 & x_n \end{bmatrix} \begin{bmatrix} w_0 \\ w_1 \end{bmatrix}$$

- Now we can calculate $\mathbf{y}$ using matrix multiplication and the "matmul" Python operator:

![](img/matmul.png)

- We're going to create a logistic regression model to classify a Pokemon as "legendary" or not
- Recall that in logistic regression we map our linear model to a probability:

$$z=\boldsymbol{w^T}\boldsymbol{x}$$

$$P(y = 1) = \frac{1}{(1+\exp(-z))}$$

- For classification purposes, we typically then assign this probability to a discrete class (0 or 1) based on a threshold (0.5 by default):

$$y=\left\{
\begin{array}{ll}
    0, & P(y = 1)\le0.5 \\
    1, & P(y = 1)>0.5 \\
\end{array} 
\right.$$ 

- Let's code that up:

In [ ]:
def sigmoid(x, w, output="soft", threshold=0.5):


- For example, if $w = [0, 0]$:

- Let's calculate the accuracy of the above model:

- Just like in the linear regression example earlier, we want to optimize the values of our weights!
- We need a loss function!
- As you discussed in 573, we use the log loss (cross-entropy loss) to optimize logistic regression
- Here's the loss function and its gradient (see [Appendix B](appendixB_logistic-loss.ipynb) if you want to learn more about these, they're actually quite simple!):
  
$$f(w)=-\frac{1}{n}\sum_{i=1}^ny_i\log\left(\frac{1}{1 + \exp(-w^Tx_i)}\right) + (1 - y_i)\log\left(1 - \frac{1}{1 + \exp(-w^Tx_i)}\right)$$

$$\frac{\partial f(w)}{\partial w}=\frac{1}{n}\sum_{i=1}^nx_i\left(\frac{1}{1 + \exp(-w^Tx_i)} - y_i\right)$$

In [ ]:
def logistic_loss(w, X, y):



def logistic_loss_grad(w, X, y):


In [ ]:
w_opt = minimize(logistic_loss, np.array([-1, 1]), jac=logistic_loss_grad, args=(X, y)).x
w_opt

- Let's check our solution against the `sklearn` implementation:

- This is what the optimized model looks like:

- Checking that against our sklearn model:

In [ ]:
lr.score(x.reshape(-1, 1), y)

- I mean, that's so cool team! We replicated the sklearn behavour from scratch!!!!
- In Lab 1 you'll actually write your own logistic regression class from scratch (including `.fit()`, `.predict()`, `.predict_proba()`, and `.score()`)
- By the way, I've been doing things in 2D here because it's easy to visualize, but let's double check that we can work in more dimensions by using `attack`, `defense` and `speed` to classify a Pokemon as `legendary` or not:

- Looks good to me!

## 8. The Lecture in Three Conjectures
<hr>

1. Loss functions and optimization algorithms are different things
    - Loss functions map the performance of your model to a number
    - Optimization algorithms find the model parameters that minimize the loss function
2. Gradient descent is an optimization algorithm, you can find others in `scipy.optimize.minimize()`
3. Gradient is simple to code up but not very efficient (it uses all the data to update weights each iteration), we'll explore a computationally cheaper method of gradient descent next lecture (stochastic gradient descent)
